In [1]:
import requests
from bs4 import BeautifulSoup

### Step1. 첫번째 페이지만 크롤링 해 보기

In [ ]:
url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=230&sid1=105&date=20190122&page=1'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [ ]:
crawl_ls = soup.select('div[class="list_body newsflash_body"] ul[class="type06_headline"] li')    # 처음 10개 기사
crawl_ls += soup.select('div[class="list_body newsflash_body"] ul[class="type06"] li')    # 다음 10개 기사

In [ ]:
url_ls = []
for crawl in crawl_ls: 
    url_ls += crawl.select('dt:nth-of-type(1) a') # 1번째 dt태그 지정

In [ ]:
for url in url_ls:
    response = requests.get(url['href'])
    soup = BeautifulSoup(response.text,'html.parser')
    
    for match in soup.findAll('script'):
        match.replace_with('')
        
    content_ls = soup.select('div[id="articleBodyContents"]')
    for content in content_ls:
        print(content.get_text().strip())

### Step2. 함수화하여 5페이지 크롤링하기

In [2]:
def get_url(page):
        url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=230&sid1=105&date=20190122&page='
        url += str(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')

        crawl_ls = soup.select('div[class="list_body newsflash_body"] ul[class="type06_headline"] li')    # 처음 10개 기사
        crawl_ls += soup.select('div[class="list_body newsflash_body"] ul[class="type06"] li')    # 다음 10개 기사

        url_ls = []
        for crawl in crawl_ls: 
            url_ls += crawl.select('dt:nth-of-type(1) a')    # 1번째 dt태그 지정

        return url_ls

In [3]:
def get_content(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')

        # script,br태그 지우기
        for match in soup.findAll('script'):
            match.replace_with('')

        content_ls = soup.select('div[id="articleBodyContents"]')

        for content in content_ls:
            f.write('{0}\n'.format(content.get_text().strip()))

In [4]:
# 파일 열기
f = open('IT_news.csv','w',encoding='utf-8')

# 5페이지까지 크롤링
for page in [1,2,3,4,5]:
    url_ls = get_url(page)
    for url in url_ls:
        get_content(url['href'])

# 파일 닫기
f.close()